In [7]:
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
text = '''Ndakaona kuti ndaitopedza nguva
ndakagara nguva ichifamba. Ndakabva
ndatosimuka very fast ndaida kuona
musikana wepanext wayimboita nyaya
naRue. Ndiye munhu chaiye aigona
kundibatsira. Ndakabva ndafunga maUSD
aVanessa aayitenga hanzi hee ndakuda
kutengawo kaHonda fit kangu.
Ndaitomupawo mamwe maZim dollar ekutenga maUSD acho. Kutaura chokwadi
Vanessa ndaimuda hangu imhepo
dzaingondibatawo. She was wife material
and very responsible. Ndakatora mari Iya ndikaiverenga. Inga

yakanga yawanda mari yemukadzi wangu.
Panga pava ne1900. Ndakanatsobisa 1000
yose ndikaisa muhomwe
ndaitonoichengetesa shamwari yangu.
Rue ndiye aifanira kutoisafira mari iyoyo.
Ndakazobisawo kamwe ka20 kuti ndigopa
musikana uya wepanext tichironga dhiri
redu. Ndichibuda panze ndakaita
lucky ndichiwanawo ari panze. Iyewo
anenge ayitovawo necrush neni.
Ndakamupa sign yekuti auye pandaiva.
Akatanga aringa ringa maside ose
achibva anyahwaira achiuya. Handina'''

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
encoded_data = tokenizer.texts_to_sequences([text])[0]
vocabulary_size = len(tokenizer.word_index) + 1

In [5]:
# Saving the tokenizer object for future use
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
sequences = []

for i in range(1, len(encoded_data)):
    sequence = encoded_data[:i+1]
    sequences.append(sequence)

sequences = np.array(pad_sequences(sequences, maxlen=6, padding='pre'))
X, y = sequences[:, :-1], sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=vocabulary_size)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)

In [9]:
from gensim.models import Word2Vec

In [10]:
# Creating word embeddings
sentences = [sentence.split() for sentence in text.split('.')]
model_gensim = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model_gensim.save("word2vec.model")

In [11]:
model_gensim = Word2Vec.load("word2vec.model")
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model_gensim.wv[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except KeyError:

        pass

In [19]:
first_model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocabulary_size, 100, input_length=5),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True)),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.LSTM(100),
  tf.keras.layers.Dense(vocabulary_size, activation='softmax')
])

first_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
second_model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocabulary_size, 100, weights=[embedding_matrix], input_length=5, trainable=False),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.LSTM(100),
  tf.keras.layers.Dense(vocabulary_size, activation='softmax')
])

second_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
# Training Model 1
first_history = first_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

# Training Model 2
second_history = second_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

Epoch 1/100
3/3 [==============================] - 8s 858ms/step - loss: 4.7016 - accuracy: 0.0000e+00 - val_loss: 4.7026 - val_accuracy: 0.0208
Epoch 2/100
3/3 [==============================] - 0s 138ms/step - loss: 4.6925 - accuracy: 0.1408 - val_loss: 4.7055 - val_accuracy: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 0s 58ms/step - loss: 4.6850 - accuracy: 0.2535 - val_loss: 4.7090 - val_accuracy: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 0s 53ms/step - loss: 4.6765 - accuracy: 0.1972 - val_loss: 4.7135 - val_accuracy: 0.0000e+00
Epoch 5/100
3/3 [==============================] - 0s 75ms/step - loss: 4.6669 - accuracy: 0.1831 - val_loss: 4.7194 - val_accuracy: 0.0000e+00
Epoch 6/100
3/3 [==============================] - 0s 85ms/step - loss: 4.6546 - accuracy: 0.1268 - val_loss: 4.7278 - val_accuracy: 0.0000e+00
Epoch 7/100
3/3 [==============================] - 0s 95ms/step - loss: 4.6385 - accuracy: 0.1408 - val_loss: 4.7403 - val_accuracy: 0

In [25]:
val_loss_first_model = first_history.history['val_loss'][-1]
val_loss_second_model = second_history.history['val_loss'][-1]

print(f"Validation Loss - First Model: {val_loss_first_model}")
print(f"Validation Loss - Second Model: {val_loss_second_model}")

Validation Loss - First Model: 9.446959495544434
Validation Loss - Second Model: 9.896809577941895


# Choose First Model

In [26]:
first_model.save("first_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
# Load the previously saved model
model =  tf.keras.models.load_model('first_model.h5')

def predict_next_words(model, tokenizer, text, num_words=1):
    for _ in range(num_words):

        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=5, padding='pre')


        predicted_probs = model.predict(sequence, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)


        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break


        text += " " + output_word

    return ' '.join(text.split(' ')[-num_words:])


# Ask the user for input
user_input = input("Input 5 Shona Words: ")


predicted_words = predict_next_words(model, tokenizer, user_input, num_words=3)
print(f"The next words might be: {predicted_words}")

Input 5 Shona Words: ndoda iwewe wega mwana wamambo
The next words might be: ndaitopedza nguva nguva
